In [1]:
%pip install weaviate-client langchain tiktoken pypdf rapidocr-onnxruntime os

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
WEAVIATE_API_KEY = os.getenv("WEAVIATE_API_KEY")

In [4]:
WEAVIATE_URL=os.getenv("WEAVIATE_URL")
# print(WEAVIATE_API_KEY, WEAVIATE_URL)

In [5]:
import weaviate
client = weaviate.Client(
    url=f"https://{WEAVIATE_URL}",
    auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)

c:\Users\antho\AppData\Local\Programs\Python\Python311\Lib\site-packages\weaviate\__init__.py:130: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])
c:\Users\antho\AppData\Local\Programs\Python\Python311\Lib\site-packages\weaviate\warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(
c:\Users\antho\AppData\Local\Programs\Python\Python311\Lib\site-packages\weaviate\warnings.py:133: DeprecationWarning: Dep005: You are using weaviate-client 

In [6]:
# import weaviate
# from weaviate.classes.init import Auth
# client = weaviate.connect_to_weaviate_cloud(
#     cluster_url=WEAVIATE_URL,
#     auth_credentials=Auth.api_key(WEAVIATE_API_KEY)
# )

In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\antho\AppData\Local\Temp\ipykernel_7500\3415835077.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\antho\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("practice.pdf", extract_images=True)
pages = loader.load()

In [9]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("practice.pdf",extract_images=True)
pages = loader.load()

In [10]:
# pages[0].page_content

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(separators="\n",chunk_size=300, chunk_overlap=30)
text = text_splitter.split_documents(pages)

In [12]:
# text

In [13]:
def clean_metadata(documents):
    for doc in documents:
        # Remove problematic metadata fields or clean them
        if doc.metadata:
            # Create a new metadata dict with only valid keys
            cleaned_metadata = {}
            for key, value in doc.metadata.items():
                # Replace dots with underscores and remove any other invalid characters
                clean_key = key.replace('.', '_').replace('-', '_')
                # Ensure key starts with a letter or underscore
                if not clean_key[0].isalpha() and clean_key[0] != '_':
                    clean_key = 'f_' + clean_key
                cleaned_metadata[clean_key] = value
            doc.metadata = cleaned_metadata
    return documents

# Clean the metadata before creating the vector store
cleaned_texts = clean_metadata(text)

from langchain_community.vectorstores import Weaviate
vector_db = Weaviate.from_documents(
    cleaned_texts,
    embeddings,
    client=client,
    by_text=False,
    index_name="RAG_PDF"
)

In [14]:
# vector_db.similarity_search("What is RAG?", k=2)[0].page_content

In [15]:
from langchain import HuggingFaceHub

In [16]:
model = HuggingFaceHub(
    huggingfacehub_api_token=os.getenv("HUGGINGFACEHUB_API_TOKEN"),
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature":1, "max_length":180}
)

C:\Users\antho\AppData\Local\Temp\ipykernel_7500\3467623277.py:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  model = HuggingFaceHub(


In [17]:
from langchain.prompts import ChatPromptTemplate
template = """
    You are a helpful assistant. Answer the following question based on the context provided.

    Context:
    {context}

    Question:
    {question}

    Answer:
    """
prompt = ChatPromptTemplate.from_template(template)

In [18]:
from langchain.llms import Ollama
llmModel = Ollama(model="gemma3:4b")

C:\Users\antho\AppData\Local\Temp\ipykernel_7500\2842173816.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llmModel = Ollama(model="gemma3:4b")


In [19]:
from langchain_core.runnables import  RunnablePassthrough
run = RunnablePassthrough()

In [20]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

In [21]:
retriever = vector_db.as_retriever()

In [22]:
rag_chain = (
    {'context':retriever,'question':run}
    | prompt
    | llmModel
    | output_parser
)

In [23]:
rag_chain.invoke('what is RAG?')

'An interactive demo of RAG models can be found at https://huggingface.co/rag/.'